In [ ]:
# Forecasting sales for the next 12 months ---

# Defining the forecasting horizon: number of future months to be predicted
future_months = 12


last_date = df['date'].max()


future_dates = pd.date_range(last_date + pd.DateOffset(months=1), periods=future_months, freq='MS')

# Creating a new DataFrame for the future periods with relevant predictor variables
# Here, 'promotion_done' and 'holidays' are specified as assumed future conditions
future_df = pd.DataFrame({
    'date': future_dates,
    'promotion_done': [1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1],  # Assumed promotion schedule
    'holidays': [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0]          # Assumed holiday indicators
})

future_df['month'] = future_df['date'].dt.month
future_df['season'] = future_df['month'].isin([4, 5, 6, 7, 8]).astype(int)
future_df['sin_month'] = np.sin(2 * np.pi * future_df['month'] / 12)
future_df['cos_month'] = np.cos(2 * np.pi * future_df['month'] / 12)

X_future = future_df[features]
X_future_scaled = scaler.transform(X_future)

with model:
    pm.set_data({"X": X_future_scaled})
    future_preds = pm.sample_posterior_predictive(trace, var_names=["mu"], random_seed=42)
#Forecast summary statistics 
mu_samples = future_preds.posterior_predictive["mu"]
mu_samples_combined = mu_samples.stack(sample=("chain", "draw")).transpose("sample", "mu_dim_0").values
forecast_samples = pd.DataFrame(mu_samples_combined, columns=future_df["date"].dt.strftime("%Y-%m"))

future_df["median"] = np.median(mu_samples_combined, axis=0)
future_df["q5"] = np.quantile(mu_samples_combined, 0.05, axis=0)
future_df["q95"] = np.quantile(mu_samples_combined, 0.95, axis=0)
future_df["mean"] = forecast_samples.mean(axis=0).values
future_df["std"] = forecast_samples.std(axis=0).values